In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime,timedelta
import yfinance as yf
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
import scipy.optimize
import mysql.connector
import pytz

In [38]:
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="1234",
  database="bolsa_valores"
)
cursor = conn.cursor()

x = pd.read_sql("SELECT * FROM cotacao", conn)
x

C:\Users\igorb\AppData\Local\Temp\ipykernel_18732\2082163194.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  x = pd.read_sql("SELECT * FROM cotacao", conn)


,DATE,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,DIVIDENDS
0,2023-02-23,ABEV3,13.210000,13.400000,13.120000,13.250000,24914000,0.0
1,2023-02-23,ALPA4,9.840000,10.130000,9.780000,9.910000,6708200,0.0
2,2023-02-23,ALSO3,17.730000,17.830000,17.559999,17.730000,1807900,0.0
3,2023-02-23,ARZZ3,76.620003,77.699997,76.169998,76.699997,873700,0.0
4,2023-02-23,ASAI3,18.830000,18.930000,18.490000,18.770000,5103300,0.0
...,...,...,...,...,...,...,...,...
2068,2023-03-23,VBBR3,13.520000,13.600000,12.770000,12.930000,11172700,0.0
2069,2023-03-23,VIIA3,2.000000,2.010000,1.760000,1.850000,109159900,0.0
2070,2023-03-23,VIVT3,38.459999,39.410000,38.290001,38.529999,2380300,0.0
2071,2023-03-23,WEGE3,39.599998,40.360001,39.529999,39.889999,6448400,0.0


In [42]:
symbol_list_ibrx = list(pd.read_csv('ibrx.csv',delimiter=";").index)
df_final = pd.DataFrame()
for ativo in symbol_list_ibrx:
    chamada_api = yf.Ticker(ativo + ".SA").history(period='36mo')
    df_hist = pd.DataFrame(chamada_api).reset_index()
    df_hist["Symbol"] = ativo
    df_hist = df_hist[["Date","Symbol","Open","High","Low","Close","Volume","Dividends"]]
    df_hist["Date"] = df_hist["Date"].dt.tz_convert(pytz.utc).dt.date
    df_hist["Date"] = df_hist["Date"].astype(str)
    df_final = pd.concat([df_hist, df_final],axis=0)

tuplas = tuple(df_final.itertuples(index=False, name=None))
query = "INSERT IGNORE INTO cotacao  VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(query, tuplas)
conn.commit()
conn.close()